In [165]:
# import needed library
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import math
import numpy as np

# train test split 
from sklearn.model_selection import train_test_split

# Kfold
from sklearn.model_selection import KFold

# regex
import re
 

In [103]:
#load iris dataset
data_iris = load_iris()
iris_X, iris_y = load_iris(return_X_y=True)
feature_iris = data_iris['feature_names']

In [104]:
#transform iris into dataframe
iris_X=pd.DataFrame(iris_X)
iris_y=pd.DataFrame(iris_y)

In [105]:
#create index so be merge
iris_X=iris_X.reset_index()
iris_y=iris_y.reset_index()

In [106]:
iris_X.rename(columns = {0:feature_iris[0],1:feature_iris[1],2:feature_iris[2],3:feature_iris[3]}, inplace = True)


In [107]:
iris_y.rename(columns = {0:'target'}, inplace = True) 

In [108]:
#merge dataset iris
iris=iris_X.merge(iris_y)

In [109]:
#drop index
iris.drop("index",axis=1,inplace=True)

In [9]:
def entropy(parsed_data, target_attribute):
    parsed_value_target = {}
    total_value_target = 0
  
    for i in parsed_data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

            total_value_target += 1
  
    log_result = 0

    for i in parsed_value_target:
        log_result += float(parsed_value_target[i])/total_value_target * math.log((float(parsed_value_target[i])/total_value_target), 2)
  
    return -1 * log_result

In [10]:
# hasn't handle after universal entropy
def information_gain(data, gain_attribute, target_attribute):
    gain_result = 0
    attribute_entropy_result = 0
    parsed_attribute_count = {}
    total_attribute_count = 0
    
    for i in data[gain_attribute]:
        if i is not None:
            if i not in parsed_attribute_count:
                parsed_attribute_count[i] = 1
            else:
                parsed_attribute_count[i] += 1
            
            total_attribute_count += 1
    
    for i in parsed_attribute_count:
        parsed_data = data.loc[data[gain_attribute]==i]
        attribute_entropy_result += float(parsed_attribute_count[i])/total_attribute_count * entropy(parsed_data, target_attribute)    

    gain_result += entropy(data,target_attribute) + (-1 * attribute_entropy_result)
    return gain_result

In [11]:
def split_information(data, gain_attribute):
    res = entropy(data, gain_attribute)
    if(res==0):
        res=0.00000001
    return res

In [12]:
def gain_ratio(data, gain_attribute, target_attribute):
    res = information_gain(data, gain_attribute, target_attribute) / split_information(data, gain_attribute)
    return res

In [13]:
def gain_ratio_continous(data, gain_attribute, target_attribute):
    res = information_gain_continous(data, gain_attribute, target_attribute)[0] / split_information(data, gain_attribute)
    return res,information_gain_continous(data, gain_attribute, target_attribute)[1]

In [14]:
def best_attribute(data,target_attribute,is_IG):
    gain_attribute = {
        'value': 0,
        'name': ''
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if is_IG:
                if information_gain(data, i, target_attribute) > gain_attribute['value']:
                    gain_attribute['value'] = information_gain(data, i, target_attribute)
                    gain_attribute['name'] = i
            else:
                if gain_ratio(data, i, target_attribute) > gain_attribute['value']:
                    gain_attribute['value'] = gain_ratio(data, i, target_attribute)
                    gain_attribute['name'] = i
                

    return gain_attribute['name']

In [15]:
import math

class Node:
    def __init__(self, attribute=None, label=None, vertex=None):
        self.attribute = attribute
        self.label = label
        self.vertex = vertex
        self.children = {}
        self.most_common_label = None
    
    def set_most_common_label(self, most_common_label):
        self.most_common_label = most_common_label
        
    def get_most_common_label(self):
        return self.most_common_label
        
    def setAttribute(self, attribute):
        self.attribute = attribute

    def setLabel(self, label):
        self.label = label
        
    def setVertex(self, vertex):
        self.vertex = vertex
  
    def addChildren(self, attributeValue, node):
        self.children[attributeValue] = node
    
    def getChildren(self):
        return self.children
    
    def getLabel(self):
        return self.label
    
    def getVertex(self):
        return self.vertex

In [16]:
def get_most_common_label(data, target_attribute):
    parsed_value_target = {}
  
    for i in data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

    most_common = {
        'value': 0,
        'name': ''
    }
    
    for i in parsed_value_target:
        if parsed_value_target[i] > most_common['value']:
            most_common['value'] = parsed_value_target[i]
            most_common['name'] = i
    
    return most_common['name']

In [17]:
def most_common_label_target(data,target_attribute):
    most_comm = None
    occ = 0;
    for i in data[target_attribute].unique():
        if data[data[target_attribute] == i].shape[0] >  occ:
            most_comm = i
            occ = data[data[target_attribute] == i].shape[0]
    return most_comm

In [18]:
def id3(data, target_attribute, is_IG):
    node = Node()
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
    
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_ = best_attribute(data,target_attribute,is_IG)
        node.setAttribute(best_attribute_)
        for i in data[best_attribute_].unique():
            node.addChildren(i,id3(data.loc[data[best_attribute_]==i],target_attribute,is_IG))
            node.set_most_common_label(most_common_label_target(data,target_attribute))        
            
    return node

In [19]:
def print_tree(node,depth):
    if node.label is not None: 
        print("    "*(depth+1) +str(node.label))
    else:
        print("    "*depth + "["+ node.attribute +"]")
        for i in node.children:
            print("----"*(depth+1) +str(i))
            print_tree(node.children[i],depth+1)        

In [20]:
def copy_tree(node):
    temp_node = Node()
    if node.label is not None: 
        temp_node.setLabel(node.label)
    else:
        temp_node.setAttribute(node.attribute)
        for i in node.children:
            temp= Node()
            temp= node.children[i]
            temp_node.addChildren(i, temp)
    return temp_node

In [21]:
def check_tree(node,data,index,result, target_attribute):
    if node.label is not None: 
        result.append(node.getLabel())
    else:
        if data.loc[index, node.attribute] is None:
            result.append(node.get_most_common_label())
        for i in node.children:
            if i==data.loc[index,node.attribute]:
                check_tree(node.children[i],data,index,result, target_attribute)

In [22]:
def pred(data,model,target_attribute):
    result = []
    for i in range(len(data)):
        check_tree(model,data[i:i+1],i,result, target_attribute)
    data = {target_attribute:result}
    return pd.DataFrame(data)
    

In [23]:
# hasn't handle after universal entropy
def information_gain_continous(data, gain_attribute, target_attribute):
    gain_result = 0
    save_boundary = -1
    min_entropy = 999999 
    data = data.sort_values(gain_attribute)
    data=data.reset_index().drop('index',axis=1)
    length_data = len(data)
    for i in range(length_data):
        if (i!=length_data-1):
            if (data.loc[i,target_attribute] != data.loc[i+1,target_attribute]):
                temp_boundary = (data.loc[i,gain_attribute] + data.loc[i+1,gain_attribute])/2
                parsed_data_upper = data.loc[data[gain_attribute]>=temp_boundary]
                parsed_data_lower = data.loc[data[gain_attribute]<temp_boundary]
                len_parsed = len(parsed_data_upper)               
                temp_entropy = float(len_parsed)/length_data * entropy(parsed_data_upper, target_attribute) + float((length_data-len_parsed))/length_data * entropy(parsed_data_lower, target_attribute)     
                if temp_entropy < min_entropy:
                    #print(gain_attribute, temp_boundary, temp_entropy)
                    min_entropy = temp_entropy 
                    save_boundary = temp_boundary

    gain_result += entropy(data,target_attribute) + (-1 * min_entropy)
    return gain_result,save_boundary

In [24]:
information_gain_continous(iris, 'petal length (cm)', 'target')

(0.9182958340544894, 2.45)

In [25]:
def best_attribute_c45(data,target_attribute,is_IG):
    gain_attribute = {
        'value': 0,
        'name': '',
        'boundary': -99999999
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if is_IG:
                if data[i].dtypes in [pd.np.dtype('float64'), pd.np.dtype('float32')]:
                    ig = information_gain_continous(data, i, target_attribute) 
                    if ig[0] > gain_attribute['value']:
                            gain_attribute['value'] = ig[0]
                            gain_attribute['name'] = i
                            gain_attribute['boundary'] = ig[1]
                else:
                    if information_gain(data, i, target_attribute) > gain_attribute['value']:
                            gain_attribute['value'] = information_gain(data, i, target_attribute)
                            gain_attribute['name'] = i
            else:
                if data[i].dtypes in [pd.np.dtype('float64'), pd.np.dtype('float32')]:
                    ig = gain_ratio_continous(data, i, target_attribute) 
                    if ig[0] > gain_attribute['value']:
                            gain_attribute['value'] = ig[0]
                            gain_attribute['name'] = i
                            gain_attribute['boundary'] = ig[1]
                else:
                    if gain_ratio(data, i, target_attribute) > gain_attribute['value']:
                            gain_attribute['value'] = gain_ratio(data, i, target_attribute)
                            gain_attribute['name'] = i

    return gain_attribute['name'],round(gain_attribute['boundary'],2)

In [26]:
best_attribute_c45(iris,'target',False)

('petal width (cm)', 0.8)

In [27]:
def c45(data, target_attribute,is_IG):
    node = Node()
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
    
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_,bound  = best_attribute_c45(data,target_attribute,is_IG)
        #print(best_attribute_)
        node.setAttribute(best_attribute_)
        node.set_most_common_label(most_common_label_target(data,target_attribute))
        if bound==-99999999:
            for i in data[best_attribute_].unique():
                node.addChildren(i,c45(data.loc[data[best_attribute_]==i],target_attribute,is_IG))
        else:
            node.addChildren('>='+str(bound),c45(data.loc[data[best_attribute_]>=bound],target_attribute,is_IG))
            node.addChildren('<'+str(bound),c45(data.loc[data[best_attribute_]<bound],target_attribute,is_IG))
    return node

In [28]:
def c45_prun(data, target_attribute,is_IG,vertex_=None):
    node = Node(vertex = vertex_)
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
     
    elif len(data.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_,bound  = best_attribute_c45(data,target_attribute,is_IG)
        #print(best_attribute_)
        node.setAttribute(best_attribute_)
        node.set_most_common_label(most_common_label_target(data,target_attribute))
        if bound==-99999999:
            for i in data[best_attribute_].unique():
                node.addChildren(i,c45_prun(data.loc[data[best_attribute_]==i],target_attribute,is_IG,i))
        else:
            node.addChildren('>='+str(bound),c45_prun(data.loc[data[best_attribute_]>=bound],target_attribute,is_IG,'>='+str(bound)))
            node.addChildren('<'+str(bound),c45_prun(data.loc[data[best_attribute_]<bound],target_attribute,is_IG,'<'+str(bound)))
    return node

In [29]:
c45(iris, "target",True)

In [30]:
print_tree(c45(iris, "target",True),0)

[petal length (cm)]
---->=2.45
    [petal width (cm)]
-------->=1.75
        [sepal width (cm)]
------------>=3.15
            [sepal length (cm)]
---------------->=6.05
                    2
----------------<6.05
                    1
------------<3.15
                2
--------<1.75
        [petal length (cm)]
------------>=4.95
            [petal width (cm)]
---------------->=1.55
                [sepal length (cm)]
-------------------->=6.95
                        2
--------------------<6.95
                        1
----------------<1.55
                    2
------------<4.95
            [petal width (cm)]
---------------->=1.65
                    2
----------------<1.65
                    1
----<2.45
        0


In [31]:
def check_tree_c45(node,data,index,result, target_attribute):
    if node.label is not None: 
        result.append(node.getLabel())
    else:
        if data.loc[index, node.attribute] is None:
            result.append(node.get_most_common_label())
        else:
            for i in node.children:
                #print(i)
                if i[0]=='<':
                    #print(i,2)
                    bound=float(i[1:])
                    if bound>data.loc[index,node.attribute]:
                        #print(data.loc[index,node.attribute])
                        check_tree_c45(node.children[i],data,index,result, target_attribute)
                elif i[0]=='>':
                    bound=float(i[2:])
                    #print(i,1)
                    if bound<=data.loc[index,node.attribute]:
                        #print(data.loc[index,node.attribute])
                        check_tree_c45(node.children[i],data,index,result, target_attribute)
                else:
                    if i==data.loc[index,node.attribute]:
                        check_tree(node.children[i],data,index,result, target_attribute)
                    

In [32]:
def pred_c45(data,model,target_attribute):
    result = []
    for i in range(len(data)):
        check_tree_c45(model,data[i:i+1],i,result, target_attribute)
    data = { target_attribute: data[target_attribute]
            ,'prediction':result}
    return pd.DataFrame(data)
    

In [33]:
p=iris

In [34]:
p=p.drop('target',axis=1)

In [35]:
p=p.reset_index().drop('index',axis=1)

In [36]:
p['sepal width (cm)'] = None

In [37]:
def accuracy(pred,data):
    cnt = 0
    for i in range(len(pred)):
        if pred.loc[i] == data.loc[i]:
            cnt+=1
    return cnt*100/len(pred)

In [38]:
def get_data_validate(data):
    data_column = data.columns
    data_10 = pd.DataFrame(columns=data_column)
    data_90 = pd.DataFrame(columns=data_column)
    #print(data_column)
    count_10 = 0
    count_90 = 0
    
    for i in range(data.shape[0]):
        if(i%10 == 1):
            # Pass the row elements as key value pairs to append() function 
            data_10 = data_10.append(data.loc[[i]] , ignore_index=True)
        else:
            data_90 = data_90.append(data.loc[[i]] , ignore_index=True)
    return data_10, data_90

In [39]:
def prune_tree(node, attribute, vertex):
    if(node.label is not None):
        return node,0
    elif(node.attribute == attribute and node.vertex == vertex):
        node.children = {}
        node.attribute = None
        node.label = node.most_common_label
        
        return node,1
    else:
        for i in node.children:
            a,b = prune_tree(node.children[i], attribute, vertex)
            
        return node,b

In [40]:
def post_pruning(data, node, current_node, children_node, target_attribute):
    if children_node == 0:
        return node
    else:
        for i in current_node.children:
            if current_node.vertex is None: 
                #print(i,1)
                post_pruning(data, node, current_node.children[i], children_node-1, target_attribute)
            else:
                if current_node.attribute is not None:
                    data10, data90 = get_data_validate(data)
                    #print(current_node.attribute,2)
                    save = copy_tree(current_node)
                    temp_node,c = prune_tree(node, current_node.attribute, current_node.vertex)
                    train=data90.drop(target_attribute,axis=1)
                    #print_tree(node,0)
                    temp_node_pred = pred_c45(train, node, target_attribute)
                    #print(temp_node_pred)
                    temp_node_accuracy = accuracy(temp_node_pred[target_attribute], data90[target_attribute])
                    #print(temp_node_accuracy)
                    if temp_node_accuracy == 100:
                        post_pruning(data, temp_node, current_node, children_node, target_attribute)
                    else :
                        
                        node.addChildren(current_node.vertex, save)
                        #print_tree(node,0)


                        post_pruning(data, node, save.children[i], children_node, target_attribute)
        return node

In [41]:





############################################          ANN              ###################################################





def sigmoid(x):
    return 1/(1+math.exp(-x))

In [42]:
def sign(x):
    if (x>0.5):
        return 1
    else:
        return 0

In [43]:
def count_error(target, output):
    return 0.5*(target-output)*(target-output)

In [44]:
class Neuron:
    def __init__(self, out=None, w=None, is_used=None, error=None):
        self.out = out
        self.w = []
        self.is_used = is_used
        self.error = 0
        self.deltaW = []
        
    def set_out(self, out):
        self.out = out
        
    def get_out(self):
        return self.out
    
    def add_deltaW(self, value):
        self.deltaW.append(value)
        
    def get_deltaW(self, index):
        return self.deltaW[index]
    
    def get_arrdW(self):
        return self.deltaW
    
    def set_deltaW(self, index, value):
        self.deltaW[index] = value
        
    def add_w(self, value):
        self.w.append(value)
        
    def set_w(self, index, value):
        self.w[index] = value
    
    def get_w(self, index):
        return self.w[index]
    
    def get_arrW(self):
        return self.w
    
    def set_is_used(self, is_used):
        self.is_used = is_used
        
    def get_is_used(self):
        return self.is_used
    
    def set_error(self, error):
        self.error = error
    
    def get_error(self):
        return self.error

In [45]:
x = [Neuron() for i in range (5)]
y = [x for i in range(5)]

In [46]:
def LenCol(arr):
    return len(arr)
def LenRow(inp,hid,out):
    return max(inp,max(hid)+1,out)
def MakeMatrix(row,col):
    return [[Neuron() for i in range (col)] for j in range(row)]

In [47]:
def printMatrixMLP(matrix):
    for i in range(len(matrix)):
        temp_matrix_out = []
        for j in range(len(matrix[i])):
            temp_matrix_out.append(matrix[i][j].get_arrW())
        print(temp_matrix_out)

In [48]:
def convTarget(output,unique):
    res = []
    for i in range(unique):
        if (i!=output):
            res.append(0)
        else:
            res.append(1)
    return res
    

In [49]:
def resetDeltaWeight(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                mat[j][i].set_deltaW(k,0)

In [50]:
def initW(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                mat[j][i].add_w(np.random.uniform(-1,1)) 
                mat[j][i].add_deltaW(0)

In [51]:
def feedforward(mat,layer):
    for i in range(1,len(layer)):
        if (i!=len(layer)-1):
            mat[layer[i]][i].set_out(1)
        for j in range(layer[i]):
            net = 0;
            for k in range(layer[i-1]):
                #print(k,i-1,j)
                net = net + mat[k][i-1].get_out()*mat[k][i-1].get_w(j)
#             if (i!=len(layer)-1):
            mat[j][i].set_out(sigmoid(net))
#             else:
#                 mat[j][i].set_out(sign(sigmoid(net)))

In [52]:
def get_error_total(matrix, n_output, target_arr):
    output_col = len(matrix[0])-1
    total_error = 0
    for i in range(n_output):
        total_error += 0.5*(target_arr[i] - matrix[i][output_col].get_out())**2
    
    return total_error

In [53]:
def set_output_error(matrix, row, col, target_value):
    output_value = matrix[row][col].get_out()
    #print(output_value, target_value)
#     print(output_value * (1 - output_value) * (target_value - output_value))
    matrix[row][col].set_error(output_value * (1 - output_value) * (target_value - output_value))

In [54]:
def set_hidden_error(matrix, row, col):
    output_value = matrix[row][col].get_out()
    delta_weight_error = 0
    for i in range(len(matrix[row][col].w)):
        delta_weight_error += matrix[row][col].get_w(i) * matrix[i][col+1].get_error()
    matrix[row][col].set_error(output_value * (1 - output_value) * delta_weight_error)

In [55]:
def update_delta_weight(matrix, row, col, idx , learning_rate):
    #print(matrix[i][col+1].get_error(),matrix[row][col].get_out(),row,col)
    #print(row,col+1)
    delta_weight = learning_rate * matrix[row][col].get_error() * matrix[idx][col-1].get_out()
    #print(delta_weight)
    
    current_weight = matrix[idx][col-1].get_deltaW(row)
    matrix[idx][col-1].set_deltaW(row, current_weight + delta_weight)
    #print(idx,col-1,row,current_weight + delta_weight)

In [56]:
def update_weight(mat,layer):
    for i in range(len(layer)-1):
        for j in range(layer[i]):
            for k in range(layer[i+1]):
                curr_weight = mat[j][i].get_w(k)
                delta_weight = mat[j][i].get_deltaW(k)
                mat[j][i].set_w(k,curr_weight + delta_weight) 

In [57]:
def backpropagation(matrix, layer, learning_rate, target_arr):
    for i in range(len(layer)-1, 0, -1):
        if i == len(layer)-1:
            for j in range(layer[i]):
                set_output_error(matrix, j, i, target_arr[j])
        else:
            for j in range(layer[i]):
                set_hidden_error(matrix, j, i)
        for j in range(layer[i-1]):
            for k in range(layer[i]):
                update_delta_weight(matrix, k, i, j, learning_rate)

In [58]:
hidden_node = [3,4]
def MLP(data,target, hidden_node, epochs, learning_rate):
    output_node = data[target].nunique()
    input_node = len(data.columns)
    hidden_node.insert(0,input_node)
    hidden_node.append(output_node)
    layer_node = hidden_node
    mlp = MakeMatrix(LenRow(input_node,hidden_node,output_node),LenCol(hidden_node))
    initW(mlp,layer_node)
    train = data.drop(target,axis=1)
    batch = len(data)/10;
    for k in range(epochs):
        error = 0
        for i in range(int(batch)):
            for j in range(10):
                for index,col in enumerate(train.columns):
                    mlp[index][0].set_out(data[col][j+i*10])
                mlp[input_node-1][0].set_out(1)
                feedforward(mlp,layer_node)
                backpropagation(mlp, layer_node, learning_rate, convTarget(data[target][j+i*10],output_node))
                error += get_error_total(mlp, len(convTarget(data[target][j+i*10],output_node)), convTarget(data[target][j+i*10],output_node))
            update_weight(mlp,layer_node)
            resetDeltaWeight(mlp,layer_node)
        print(error)
    print()
    print()
    printMatrixMLP(mlp)

    return mlp,layer_node

In [59]:
def predMyMLP(data, target, model, output_layer):
    out = []
    mlp = model[0]
    for i in range(0,data.shape[0],1):
        for index,col in enumerate(data.columns):
            mlp[index][0].set_out(data[col][i])
        mlp[len(data.columns)-1][0].set_out(1)
        feedforward(mlp,model[1])
        output_col = len(mlp[0])-1
        #print(output_col)
        mx=0
        res=-1
        for j in range(output_layer):
            # print(mlp[j][output_col].get_out())
            if mlp[j][output_col].get_out()>mx:
                mx = mlp[j][output_col].get_out()
                res = j
        # print()
        out.append(res)
    data = { target: data[target],
             'prediction':out}
    return pd.DataFrame(data)


# Testing

In [61]:
class_list = iris['target'].unique()

# Create confusion matrix from testing result of a model
#
# Input:
#    pandas.DataFrame   prediction_data
#    String             actual_column
#    String             prediction_column
#    Array              class_list
# Output:
#    pandas.DataFrame   confusion_matrix
#
def confusion_matrix(prediction_data, actual_column, prediction_column, class_list):
    actual_data = pd.Categorical(prediction_data[actual_column], categories=class_list)
    prediction_data = pd.Categorical(prediction_data[prediction_column], categories=class_list)
    confusion_matrix = pd.crosstab(actual_data, prediction_data, rownames=['Actual'], colnames=['Predicted'], dropna=False)
    return confusion_matrix

In [62]:
# Get accuracy from testing result of a model
#
# Input:
#    pandas.DataFrame   prediction_data
#    String             actual_column
#    String             prediction_column
# Output:
#    float              accuracy
#
def testing_accuracy(prediction_data, actual_column, prediction_column):
    count = 0
    for index, row in prediction_data.iterrows():
        if(row[actual_column] == row[prediction_column]):
            count = count + 1
    return round(count/prediction_data.shape[0],2)
            


## Train Test Split

In [112]:
iris_X.drop('index',axis=1,inplace=True);
iris_y.drop('index',axis=1,inplace=True);

In [182]:
################################# Testing DTL
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size=0.1, random_state=42)

In [183]:
def mergeData(X_train, X_test, y_train, y_test):
    X_train = X_train.reset_index()
    y_train = y_train.reset_index()
    X_test = X_test.reset_index()
    y_test = y_test.reset_index()
    trainData = X_train.merge(y_train)
    testData = X_test.merge(y_test)
    trainData.drop("index",axis=1,inplace=True)
    testData.drop("index",axis=1,inplace=True)
    return trainData,testData

In [184]:
trainData,testData = mergeData(X_train, X_test, y_train, y_test)

In [185]:
model = c45(trainData, "target",True)
prediction_dtl = pred_c45(testData, model, "target")
print(prediction_dtl)

    target  prediction
0        1           1
1        0           0
2        2           2
3        1           1
4        1           1
5        0           0
6        1           1
7        2           2
8        1           1
9        1           1
10       2           2
11       0           0
12       0           0
13       0           0
14       0           0


In [186]:
print(confusion_matrix(prediction_dtl,'target','prediction', class_list))

Predicted  0  1  2
Actual            
0          6  0  0
1          0  6  0
2          0  0  3


In [150]:
testing_accuracy(prediction_dtl,'target','prediction')

1.0

In [190]:
hidden_node = [3,4]
x = MLP(trainData, 'target', hidden_node,100,0.1)

46.876436791309445
45.272112358356
44.809170683155536
44.57595675737858
44.31045511394885
43.74798788709486
42.50525457624772
41.01423859703754
39.15093955106138
36.892342222871
34.41933457341722
32.00211141838871
29.884096399931387
28.195496196120438
26.925222233538967
25.989880177371166
25.301242724357078
24.787405223836426
24.395979570437206
24.091003523354534
23.848144856503982
23.650796627230566
23.487411821335073
23.349782017868325
23.231937423903595
23.129436357538808
23.038895500049836
22.957668786408473
22.883617817956758
22.814938204486555
22.750021295935046
22.687346352716315
22.62542301807202
22.562845233059118
22.498558397359304
22.43238312663996
22.36549511828245
22.30005243877334
22.23758450424558
22.177503859911297
22.117321720646878
22.054205266021256
21.986704507809375
21.915899484999397
21.84503115086337
21.77712836207869
21.711689387718874
21.642179709164125
21.557543784103988
21.452414592537583
21.33969350504342
21.246300276148755
21.199539266488905
21.230468323073

In [191]:
prediction_ann = predMyMLP(testData, 'target', x, 3)
print(prediction_ann)

    target  prediction
0        0           0
1        0           0
2        0           0
3        0           0
4        0           0
5        1           2
6        1           2
7        1           2
8        1           2
9        1           1
10       1           2
11       2           2
12       2           2
13       2           2
14       2           2


In [192]:
print(confusion_matrix(prediction_ann,'target','prediction', class_list))

Predicted  0  1  2
Actual            
0          5  0  0
1          0  1  5
2          0  0  4


In [193]:
testing_accuracy(prediction_ann,'target','prediction')

0.67

## CrosVal

# DTL

In [161]:
kf = KFold(n_splits=10,shuffle=True)

In [195]:
accuracy_DTL_KF = []
for train_index, test_index in kf.split(iris_X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = iris_X.loc[train_index], iris_X.loc[test_index]
    y_train, y_test = iris_y.loc[train_index], iris_y.loc[test_index]
    trainData,testData =  mergeData(X_train, X_test, y_train, y_test)
    model = c45(trainData, "target",True)
    prediction_dtl = pred_c45(testData, model, "target")
    accuracy_DTL_KF.append(testing_accuracy(prediction_dtl,'target','prediction'))

In [196]:
accuracy_DTL_KF

[0.87, 0.93, 1.0, 1.0, 0.8, 0.93, 0.93, 0.87, 0.93, 1.0]

In [197]:
round(np.mean(accuracy_DTL_KF)*100,2)

92.6

# MLP

In [198]:
accuracy_MLP_KF = []
for train_index, test_index in kf.split(iris_X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = iris_X.loc[train_index], iris_X.loc[test_index]
    y_train, y_test = iris_y.loc[train_index], iris_y.loc[test_index]
    trainData,testData =  mergeData(X_train, X_test, y_train, y_test)
    hidden_node = [3,4]
    model = MLP(trainData, 'target', hidden_node,1000,0.1)
    prediction_ann = predMyMLP(testData, 'target', x, 3)
    accuracy_MLP_KF.append(testing_accuracy(prediction_ann,'target','prediction'))

47.559908720059845
45.4656085019654
44.83516605228824
44.53019071895165
44.26185598035731
43.939272205244485
43.519638386263814
42.97283353513247
42.27337225447556
41.40368333430626
40.36059764806955
39.15973362147148
37.83551800611511
36.43675704258144
35.02008109700589
33.64179439658697
32.34833498955947
31.169749602834326
30.119464855400924
29.198060193261806
28.397787635109978
27.706474818430838
27.110399300420195
26.59608467947846
26.151230941318516
25.76507554266825
25.428432034831854
25.13357036091783
24.874034853999106
24.64445078546334
24.44034349021984
24.257979352727006
24.09423038525268
23.94646056496276
23.812430593988545
23.690217199527506
23.578142867227957
23.474711623570236
23.378545916664176
23.28831871915822
23.202674123843956
23.120131138202993
23.038977175043417
22.957204433191404
22.87267430019749
22.783868730114786
22.691181559306518
22.597329197121056
22.50602765395013
22.4205105535363
22.34278021844404
22.273431328775633
22.211997193571484
22.157506710430894
22

48.60419142331504
45.68804745820104
44.4221058285292
43.6556105168813
42.998361037143724
42.286661300594915
41.389999808790456
40.11107951482079
38.91963790392935
37.62352434343999
36.254759301064354
34.90784099200088
33.63136744530516
32.460635265588984
31.415287915435314
30.499531688615892
29.706201226748988
29.022020765552007
28.431816024269654
27.920970244212917
27.476492322769342
27.087294006735224
26.744105142058363
26.439256478253228
26.166434173264886
25.920447869785292
25.69702676288699
25.492646808245695
25.304387728144416
25.129816618864645
24.966894249096484
24.813900150180448
24.669373098041387
24.53206422099403
24.400900413442127
24.274955876940922
24.153429524344254
24.035625835638342
23.920936761160235
23.80882270307432
23.69879172245059
23.590377893290878
23.483121545407602
23.376554867934647
23.27019513896155
23.163545227854684
23.056098774893456
22.947346985664762
22.83678508363478
22.723918053391838
22.608266622625948
22.489375373264814
22.366825711122477
22.2402574

14.764190747863053
13.05803492045332
11.439377059656197
14.086616704692853


[[1.1674276157114976, -0.4114296914058166, 0.29666662928369547], [0.17520691868961394, 0.2501825169609848, -4.578829985776212, 1.2770738965597312], [-5.364676773281209, 1.458398537015527, 2.029714965103626], []]
[[3.011845280190242, -0.631484033485076, 1.79425849575142], [-0.4716566383421418, -0.2406360994650022, 0.4560401450277352, 0.6917407451601746], [3.7680179546692547, -3.123972971595474, -4.592499209580324], []]
[[-3.620198376965274, -1.0833482279159807, -2.961710898626113], [-2.5688443995739223, 2.4195287742974316, -1.1444749427777112, -3.2447321146409935], [-4.390406262271283, -2.7351775227704724, 4.308219065464194], []]
[[-3.334879421696121, 0.14971173945519536, -0.26907424524753315], [], [-3.001116567529926, 2.7666285561619017, 0.12838063542780895], []]
[[0.6064466519967557, 0.7896243664728377, 0.610122730515797], [], [], []]
[[], [], [], []]
50.33170387838623
46.001296366101464
45.161639817860554
44

22.11570564530824
8.863003188287038
21.664681008590655
8.647682683562092
21.23224296999273
8.43466632396959
20.65735482963413
8.191261801598234
20.143725671924663
8.00608005415933
19.689782631373404
7.85337769546186
19.333970814344216
7.665392387528187
18.920841843215015
7.476449764264201
18.499542793374502
7.2381912998409454
18.277370689900003
7.001624782944916
17.39065809975895
6.687527602789978
16.303031826648972
6.375302539408934
15.861944358113199
6.229295316496073
16.53257272233706
17.07791459573042
6.000560875537212
11.376427840774257
15.381745568330398
15.683448802369998
6.274793324684594
16.086759637185803
6.243353911925252
16.730360762137725
6.165927268967141
16.282769854081973
6.214833331724478
16.721323994407022
6.084302350613606
15.534978684563312
6.148060461729908
15.815032747107756
5.109635798351649
11.761999127274398
15.191854517322342
11.58009956797268
10.720315701886094
11.470201427028364
4.951680864592818
11.57917868074048
12.02841831605775
8.446375922212825


[[-4.8

19.24640164241485
17.88649174541724
20.945267957708076
18.440086118355953
18.08557083761076
18.605192048651286
17.255816889817623
19.01302874341598
16.08156089733525
22.872369178031324
13.514627077780327
22.20779064852998
13.18487158040156
21.683777490488318
12.444658342125436
22.65223429856
12.017434421459964
22.232047673673154
11.540882640562868
21.696633458865474
10.990776172241445
21.20238226882425
10.504169177376516
20.774958398763676
10.06917057601523
20.249189786219556
9.620901652659711
19.614099575251664
9.312124845683368
18.537729120178415
9.085414135542733
17.320338506640965
8.29658751192705
16.23875632222366
7.936389444777139
15.883593844460517
7.73632373737646
15.29788705014068
7.228226053468391
14.430637732847948
6.837897263795141
14.060862265573288
6.60352668699707
13.756126814573696
6.3894039753543606
13.32915589769037
6.174049481440909
12.443822605010922
5.972884983646916
12.486928564296157
5.821858907552611
11.999238261725269
5.578866467815106
12.690780079954767
5.4626

23.864286246544513
23.3894967247095
24.525972313392835
24.228436964524747
23.9273782582217
24.309126042958773
22.780832610739886
23.302306266988083
22.479985266029136
23.44716257072495
23.56064685092157
22.92656664574393
22.180439700761106
23.85592386274797
25.42031752955612
27.814690246625286
23.79196939932206
22.97813078298134
22.949214919959736
23.61226347159752
22.85309750607843
22.94053524964796
23.24414086304079
22.796961136082228
23.36953714020086
22.667167827973525
22.7540401551399
23.36911562056361
22.660222671698875
22.67101917553696
23.21123174560903
22.529105188354457
22.607836499934525
23.25142899164689
22.619916900170036
22.53993658029181
22.97839368611746
22.467302312327014
22.48567572131441
22.51117181520439
23.220496348154274
22.71952924547182
22.374546922842732
22.46518989937028
22.470814472789964
23.114020169395836
22.627633851923594
22.31557408000769
22.484128431879
22.390938815389582
23.024648181764057
22.5297379501279
22.272916763558502
22.611259005375405
22.29491

In [199]:
accuracy_MLP_KF

[0.73, 0.53, 0.73, 0.6, 0.8, 0.73, 0.67, 0.67, 0.6, 0.73]

In [201]:
round(np.mean(accuracy_MLP_KF)*100,2)

67.9